# Advent of Code 2021

Imports

In [100]:
import math

import numpy as np
import pandas as pd

---
## Day 1
#### Part 1

In [101]:
# get puzzle inputs
with open('data/2021_day_1_input.txt', 'r') as f:
    data = [int(ln) for ln in f.readlines()]

In [102]:
def count_increasing_depths(data):
    arr = np.array(data)
    return (arr[:-1] < arr[1:]).sum()

print(count_increasing_depths(data))

1195


#### Part 2

In [103]:
def window_sum(data, window):
    df = pd.DataFrame(data, columns=['depth'])
    return df.depth.rolling(window).sum().tolist()

print(count_increasing_depths(window_sum(data, 3)))

1235


---
## Day 2
#### Part 1

In [104]:
# get puzzle inputs
with open('data/2021_day_2_input.txt', 'r') as f:
    data = [ln.strip() for ln in f.readlines()]

In [105]:
def directional_command_to_xy(s):
    direction, units = tuple(s.split())
    
    # increase horizontal position
    if direction == 'forward':  
        xy = np.array([int(units), 0])
    
    # increase depth
    elif direction == 'down':  
        xy = np.array([0, int(units)])
    
    # decrease depth
    elif direction == 'up':
        xy = np.array([0, -int(units)])
        
    else:
        print(f'Unknown direction command: {direction}')
        xy = None
        
    return xy

def final_position(data):
    return np.array([directional_command_to_xy(s) for s in data]).sum(axis=0)

print(math.prod(final_position(data)))

1459206


#### Part 2